In [ ]:
from pypdf import PdfReader
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

def extract_activities_from_text(text, year, month):
    # pattern = f"^(\d{{2}}\/\d{{2}})\s+(.*?)\s+((-?\d{{1,3}}(,\d{{3}})*)*\.\d{{2}})"
    pattern = r"^(\d{2}\/\d{2})\s+(.*?)\s+((-?\d{1,3}(,\d{3})*)*\.\d{2})"

    lines = []
    for line in text.split("\n"):
        match = re.match(pattern,line) 
        if match:
            if match.group(1)[:2] == "12" and month == "01":
                this_year = str(int(year) - 1)
            else:
                this_year = year
            lines.append(f"{match.group(1)}/{this_year},{match.group(2)},{match.group(3).replace(',','')}")
    return "\n".join(lines)

# year = os.path.basename(pdf_path).split("-")[0][:4] #.split(".")[0]
# text_data = extract_text_from_pdf(pdf_path)
# activities = extract_activities_from_text(text_data, year)

directory_path = "./data"
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
activities = "Transaction date,Description,Amount"
for pdf in pdf_files:
    year = os.path.basename(pdf).split("-")[0][:4]
    month = os.path.basename(pdf).split("-")[0][4:6]
    text_data = extract_text_from_pdf(pdf)
    activities += "\n" + extract_activities_from_text(text_data, year, month)



In [ ]:
with open("Chase-2018010-20231231.csv", "w") as file:
    file.write(activities)
